In [28]:
##### Import package###
import numpy as np
import matplotlib
import matplotlib.pylab as plt
from scipy.integrate import odeint

from pylab import *
#import pylab as p
from scipy import integrate
import ipywidgets as widgets
from ipywidgets import (
    interactive,
    interact,
    fixed,
    FloatRangeSlider,
    FloatSlider,
    interactive_output,
    interact_manual,
    Button,
    ToggleButton

)
%matplotlib inline


###################################
#####Main function######

LTrajectory = []  # Defining the list of trajectories


def LotKaVoltInteract(K1, K2, r1, alpha21, r2, alpha12, Vectors, Xo_abs, Yo_abs):
    global X, fig, Block, X0
    # The two function defining the system and the system

    def f(x, y):
        f = r1*x*(1-(x+alpha21*y)/K1)
        return f

    def g(x, y):
        g = r2*y*(1-(y+alpha12*x)/K2)
        return g

    def dX_dt(X, t=0):
        return array([f(X[0], X[1]), g(X[0], X[1])])
 # Maximal value of the two coordinates
    xmax = 1.03*max(K1, K2/alpha12) if alpha12 != 0 else 1.03*K1
    ymax = 1.03*max(K2, K1/alpha21) if alpha21 != 0 else 1.03*K2

    fig = plt.figure(figsize=(15, 12))  # The figure
    plt.title('Espace des phases, isoclines, points fixes et trajectoires')
    plt.xlabel('Population 1', fontsize=15)
    plt.ylabel('Population 2', fontsize=15)
    plt.xlim(-0.05*xmax, xmax)
    plt.ylim(-0.05*ymax, ymax)
    ########## The vector field ##########
    cm = matplotlib.cm.copper_r
    norm = matplotlib.colors.Normalize()
    #sm = matplotlib.cm.ScalarMappable(cmap=cm, norm=norm)
    # sm.set_array([])

    nb_points = 15
    x = np.linspace(0, xmax, nb_points)
    y = np.linspace(0, ymax, nb_points)
    X1, Y1 = meshgrid(x, y)
    DX1, DY1 = dX_dt([X1, Y1])
    M = (hypot(DX1, DY1))
    M[M == 0] = 1.
    DX1 = DX1/M
    DY1 = DY1/M

    Q = plt.quiver(X1, Y1, DX1, DY1, norm(log(M)), cmap=cm,
                   zorder=10, visible=not Vectors, pivot='tip')
    cbar = plt.colorbar(ticks=[0.05, 0.95])
    cbar.ax.set_yticklabels(['Slow evolution', 'Fast evolution'], fontsize=15)
    # the two isoclines
    plt.plot([K1, 0], [0, K1/alpha21],
             '#dede00', lw=3) if alpha21 != 0 else plt.plot([K1, K1], [0, ymax], '#dede00', lw=3)
    plt.plot([0, 0], [0, ymax], '#dede00', lw=3)
    plt.plot([K2/alpha12, 0], [0, K2],
             '#377eb8', lw=3) if alpha12 != 0 else plt.plot([0, xmax], [K2, K2], '#377eb8', lw=3)
    plt.plot([0, xmax], [0, 0], '#377eb8', lw=3)

    # The points of equilibrium
    plt.scatter(0, 0, c='black', s=100, zorder=10)
    if 1-alpha12*alpha21 != 0:
        xinter = (K1-alpha21*K2)/(1-alpha12*alpha21)
        yinter = (K2-alpha12*K1)/(1-alpha12*alpha21)
        if xinter >= 0 and yinter >= 0:
            plt.scatter(xinter, yinter, c='black', s=100, zorder=10)
    # The capacities
    plt.scatter(0, K2, c='black', s=100, zorder=10)
    plt.scatter(K1, 0, c='black', s=100, zorder=10)
    # Initial state
    Xo = Xo_abs*xmax/200
    Yo = Yo_abs*ymax/200
    plt.scatter(Xo, Yo, c='black', s=30, zorder=10)
    ####################

    ######## xticks, yticks
    if alpha12 != 0:
        plt.xticks([0, K1, K2/alpha12], ['0', r'$K_1$',
                   r'$K_2/\alpha_{12}$'], fontsize=16) if K1 != K2/alpha12 else plt.xticks([0, K1], ['0', r'$K_1=K_2/\alpha_{12}$'], fontsize=15)
    if alpha21 != 0:
        plt.yticks([0, K2, K1/alpha21], ['0', r'$K_2$',
                   r'$K_1/\alpha_{21}$'], fontsize=15) if K2 != K1/alpha21 else plt.yticks([0, K2], ['0', r'$K_2=K_1/\alpha_{21}$'], fontsize=15)

    if alpha12 == 0:
        plt.xticks([0, K1], ['0', r'$K_1$'], fontsize=15)
    if alpha21 == 0:
        plt.yticks([0, K2], ['0', r'$K_2$'], fontsize=15,)

    # Computing and drawing a trajectory
    t = linspace(0, 100,  1000)
    X0 = array([Xo, Yo])
    X = integrate.odeint(dX_dt, X0, t)
    plt.plot(X[:, 0], X[:, 1], '#f781bf', lw=3.5, zorder=8)

    # Drawing trajectories in the list
    for T in LTrajectory:
        plt.plot(T[0], T[1], '#f781bf', lw=3.5, zorder=8)


#######################
# Defining the widgets and Layout
#################################

# Common layout of button and slider
curseur1Layout = widgets.Layout(
    width='300px', height='30px', justify_contents='flex-start')
# Common layout of button and slider
buttonLayout = widgets.Layout(
    width='200px', height='30px', justify_contents='flex-start')
# Defining Slider and button
# Slider for systeme parameters

# init
Xo_abs = widgets.IntSlider(min=0, max=200, value=198, description=r'$x_0$', readout=False,
                           continuous_update=False, layout=widgets.Layout(width='80%', height='30px'))
Yo_abs = widgets.IntSlider(min=0, max=200, value=198, description=r'$y_0$', orientation='vertical',
                           readout=False, continuous_update=False, layout=widgets.Layout(width='15px', height='85%'))
# Buttons

K1 = widgets.FloatSlider(min=1, max=12, value=3,
                         description=r'$K_1$', layout=curseur1Layout)
K2 = widgets.FloatSlider(min=1, max=12, value=3,
                         description=r'$K_2$', layout=curseur1Layout)
r1 = widgets.FloatSlider(min=0.1, max=5, value=2,
                         description=r'$r_1$', layout=curseur1Layout)
r2 = widgets.FloatSlider(min=0.1, max=5, value=1,
                         description=r'$r_2$', layout=curseur1Layout)
alpha21 = widgets.FloatSlider(
    min=0, max=10, value=0.6, step=0.01, description=r'$\alpha_{21}$', layout=curseur1Layout)
alpha12 = widgets.FloatSlider(
    min=0, max=10, value=0.5, step=0.01, description=r'$\alpha_{12}$', layout=curseur1Layout)
Vectors = widgets.ToggleButton(
    value=False, description='Click for hide/show vectors', button_style='info', layout=buttonLayout)
keep = Button(description="keep trajectory",
              button_style='info', layout=buttonLayout)
EraseAll = Button(description="Erase all",
                  button_style='info', layout=buttonLayout)
EraseLast = Button(description="Erase last trajectory",
                   button_style='info', layout=buttonLayout)
Save = Button(description="Save figure",
              button_style='info', layout=buttonLayout)
#Fix= Button(description="Fix",button_style='info',layout=buttonLayout)
Release = Button(description="Unfreeze Cursors.",
                 button_style='info', layout=buttonLayout)
##############
# Function on click


def on_keep_clicked(_):
    global LTrajectory, K1, K2, alpha12, alpha21, r1, r2, u
    LTrajectory.append([X[:, 0], X[:, 1]])
    for u in [K1, K2, alpha12, alpha21, r1, r2]:
        u.disabled = True


def on_erase_clicked(_):
    global LTrajectory
    LTrajectory = []
    Xo_abs.value = Xo_abs.value+1


def last_erase_clicked(_):
    global LTrajectory
    if len(LTrajectory) > 0:
        LTrajectory = LTrajectory[:len(LTrajectory)-1]
    Xo_abs.value = Xo_abs.value+1


def SaveTheFigure(_):
    fig.savefig('fig.pdf', dpi=600)


def Block_Sliders(_):
    global K1, K2, alpha12, alpha21, r1, r2, u
    for u in [K1, K2, alpha12, alpha21, r1, r2]:
        u.disabled = True


def Release_Sliders(_):
    global K1, K2, alpha12, alpha21, r1, r2, u
    for u in [K1, K2, alpha12, alpha21, r1, r2]:
        u.disabled = False


# Fix.on_click(Block_Sliders)
Release.on_click(Release_Sliders)
# Fix.observe(Block_Sliders,names='value')
# Actions on click
keep.on_click(on_keep_clicked)
EraseLast.on_click(last_erase_clicked)
EraseAll.on_click(on_erase_clicked)
Save.on_click(SaveTheFigure)
# Ouput the figure
main = widgets.interactive_output(LotKaVoltInteract, {'K1': K1, 'K2': K2, 'r1': r1, 'r2': r2,
                                                      'alpha21': alpha21, 'alpha12': alpha12,
                                                      'Vectors': Vectors, 'Xo_abs': Xo_abs, 'Yo_abs': Yo_abs})

# add Xo,Yo controls

main_with_XoYo=widgets.HBox([Yo_abs,widgets.VBox([main,Xo_abs])])
# Panel control
Wid_left1 = widgets.VBox([K1, K2, r1, r2, alpha21, alpha12])
Wid_left2 = widgets.VBox([Vectors,
                          Release, keep, EraseLast, EraseAll, Save])

# The final layout

Final = widgets.HBox([main_with_XoYo, widgets.VBox([Wid_left1, Wid_left2])])
Final

## Le modèle de Lotka-Volterra compétitif
Ce notebook permet de visualiser l'espace des phases et les trajectoires du modèle de Lotka-Volterra compétitif. Rappelons que ce modèle est un système d'équations différentielles linéaires modélisant l'évolution de deux populations en interaction :

$$
\left\{\begin{matrix}
\displaystyle x'=r_1x\left(1-\frac{x+\alpha_{21} y}{K_1}\right)\\
\\
\displaystyle y'= r_2y\left(1-\frac{y+\alpha_{12}x}{K_2}\right)
\end{matrix}
\right.
$$

Les deux populations suivent un modèle logistique de constante $(r_1,K_1)$  pour la première population dont la taille est donnée par la fonction $x$ et  $(r_2,K_2)$ pour la deuxième population dont la taille est donnée par la fonction $y$. Ces constantes sont ici supposées strictement positives. Les deux constantes $\alpha_{21}$ et $\alpha_{12}$ mesurent l'interaction d'une espèce sur l'autre. Dans ce notebook les interactions sont défavorables aux deux espèces, c'est à dire que les coefficients $\alpha_{21}$ et $\alpha_{12}$ sont positifs ou nuls. 

Par défaut, la figure affiche les isoclines, les points fixes, les vecteurs tangents et une trajectoire dont l'état initial est marqué par un point noir. La norme des vecteurs varie beaucoup ; nous avons donc choisi de les représenter après les avoir normés. Leur norme réelle est encodée par la couleur de la flèche : foncée si la trajectoire est rapide en ce point là, claire dans le cas contraire. 

Le point représentant la condition intiale peut être déplacé en utilisant le curseur horizontal $x_0$ sous la figure et le curseur vertical $y_0$ à gauche de la figure. Avec l'aide des curseurs à droite de la figure, on peut faire   varier tous les paramètres du système. À propos des curseurs, une astuce : si le bouton du curseur est activé, on peut le bouger "pas à pas" en utilisant les flèches du clavier.  

Un fois que l'on a obtenu un système qui nous convient, on peut enregister en mémoire les trajectoires avec le bouton adéquat.
Attention : quand une trajectoire est enregistrée, les curseurs modifiant le système sont désactivés, ils peuvent être réactivés par le bouton *Unfreeze Cursors*. 
On peut avec deux autres boutons effacer la dernière trajectoire ou bien toutes les trajectoires. Et pour finir, en cliquant sur le bouton *Save the figure*, on peut enregistrer la figure, on trouvera la figure sous format .pdf dans le répertoire à partir duquel a été lancé le notebook (cette fonctionalité n'est pas disponible en ligne). 

Le point de départ de ce notebook est un programme statique concernant le système Lotka-Volterra classique disponible sur ce [site](http://scipy.github.io/old-wiki/pages/Cookbook/LoktaVolterraTutorial). 
Je remercie Tanguy Lefort pour sa disponibilité et ses conseils sur le package *Interact* ainsi que Joseph Salmon pour ses judicieuces suggestions. 

